# File systems proxies

This notebook demonstrates file system proxies to folders and directories, which are basically strings (paths) with additional methods for creation, opening moving, deleting, renaming, linking and browsing. Three file systems are currently supported:
 * Local file system
 * HDF5 file system
 * Nexus file system

In [1]:
from spectrocrunch.io import fs,localfs,h5fs,nxfs

This notebook will create directories and files in a context manager that cleans up everything afterwards:

In [2]:
from contextlib import contextmanager
@contextmanager
def temppath(hdf5=False,nexus=False):
    with localfs.Path('.').temp() as root:
        root.mkdir()
        if hdf5:
            root = h5fs.Path(root['test.h5']).mkdir()
            print('HDF5 root: {}'.format(root))
        elif nexus:
            root = nxfs.Path(root['test.nx']).mkdir()
            print('Nexus root: {}'.format(root))
        else:
            print('Local root: {}'.format(root))
        yield root

This context manager will be used as follows:

In [3]:
for hdf5 in [True,False]:
    with temppath(hdf5=hdf5) as root:
        root.ls()
    root.ls()
    print('')

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/6WpqOE/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/6WpqOE/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/6WpqOE/test.h5:/ (does not exist)

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RLJisw
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RLJisw
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RLJisw (does not exist)



## Proxy

A proxy to an existing or non-existing path (file or directory) can be created from a string:

In [4]:
import os
with temppath() as root:
    path = os.path.join(root.path,'subdir')
    print('String: '+path)
    proxy = localfs.Path(path)
    print('Local proxy: '+str(proxy))
    assert(not proxy.exists)
    print('')
    
    path = os.path.join(root.path,'test.h5:/subdir')
    print('String: '+path)
    proxy = h5fs.Path(path)
    print('HDF5 proxy: '+str(proxy))
    assert(not proxy.exists)

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/qTEsuW
String: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/qTEsuW/subdir
Local proxy: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/qTEsuW/subdir

String: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/qTEsuW/test.h5:/subdir
HDF5 proxy: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/qTEsuW/test.h5:/subdir


### Create directories

You can use the proxy's "mkdir" method to create a directory (can be done recursively)

In [5]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        subdir = root['dir']['subdir']
        print('Device: {}'.format(subdir.device))
        print('Path: '+subdir.path)
        print('Location: '+subdir.location)
        assert(not subdir.exists)
        subdir.mkdir(recursive=True)
        assert(subdir.exists)
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/10VENf
Device: 
Path: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/10VENf/dir/subdir
Location: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/10VENf/dir/subdir

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/aW4GMu/test.h5:/
Device: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/aW4GMu/test.h5
Path: /dir/subdir
Location: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/aW4GMu/test.h5:/dir/subdir



### Browse directories

Browse directories that not necessarily exist by indexing or iteration:

In [6]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        print('Indexing:')
        path = root['dir']['subdir']
        assert(path['.'] == path)
        assert(path['..'] == root['dir'])
        assert(path['..'] == path.parent)
        assert(path == path['..']['subdir'])

        print('Iterate up from {}:'.format(path.path))
        for p in path.iterup:
            print p
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/R1PC89
Indexing:
Iterate up from /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/R1PC89/dir/subdir:
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/R1PC89/dir/subdir
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/R1PC89/dir
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/R1PC89
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev
/mntdirect/_data_id21_inhouse/wout/dev
/mntdirect/_data_id21_inhouse/wout
/mntdirect/_data_id21_inhouse
/mntdirect
/

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/MxY2In/test.h5:/
Indexing:
Iterate up from /dir/subdir:
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/MxY2In/test.h5:/dir/subdir
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/MxY2In/test.h5:/dir
/mntdirect/_data_id21_inhou

Browse directories that exist using the "ls" method or iteration:

In [7]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir']['subdir'].mkdir()
        for subdir in ['dira','dirb']:
            path[subdir].mkdir()
        
        print('Iterate {}:'.format(path.path))
        for subdir in path:
            print(subdir)
        
        print('ls:')
        root.ls(recursive=True)
        print('')


Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ic5rVE
Iterate /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ic5rVE/dir/subdir:
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ic5rVE/dir/subdir/dirb
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ic5rVE/dir/subdir/dira
ls:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ic5rVE
  |
  +-dir
    |
    +-subdir
      |
      +-dira
      |
      +-dirb

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/4QvhqW/test.h5:/
Iterate /dir/subdir:
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/4QvhqW/test.h5:/dir/subdir/dira
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/4QvhqW/test.h5:/dir/subdir/dirb
ls:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/4QvhqW/test.h5:/
  |
  +-dir
    |
    +-subdir
      |
      +-dira
      |
      +-dirb



### Create files

In [8]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        if hdf5:
            data = range(10)
        else:
            data = 'Hello World'
        path = root['file1.txt'].mkfile(data=data)
        root.ls(stats=True)
        
        with path.open(mode='r') as fileio:
            print('IO object: {}'.format(type(fileio)))
        content = path.read()
        print('Content: {}'.format(content))
        if hdf5:
            assert(tuple(data)==tuple(content))
        else:
            assert(data==content)
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/SCVbS3
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/SCVbS3
  | @size = 4096
  | @blocks = 0
  | @ctime = 2018-12-05 12:02:31.425532
  | @rdev = 0
  | @dev = 162
  | @nlink = 2
  | @gid = 1621
  | @blksize = 1048576
  | @mtime = 2018-12-05 12:02:31.425532
  | @uid = 80528
  | @atime = 2018-12-05 12:02:31.426040
  | @ino = 340637107
  | @mode = 16893
  |
  +-file1.txt
      @size = 11
      @blocks = 0
      @ctime = 2018-12-05 12:02:31.426144
      @rdev = 0
      @dev = 162
      @nlink = 1
      @gid = 1621
      @blksize = 1048576
      @mtime = 2018-12-05 12:02:31.426170
      @uid = 80528
      @atime = 2018-12-05 12:02:31.425532
      @ino = 340637123
      @mode = 33204
IO object: <type 'file'>
Content: Hello World

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/ErGiGO/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/ErGiGO/te

### Move/rename

Moving files or directories can be done with the "move" method (aliases: "mv" and "rename"). The destination can be a proxy or a string.

In [9]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir1'].mkdir()['test1.txt'].mkfile(data='Hello World')
        
        print('\nMove test1.txt to test2.txt:')
        path = path.move('test2.txt')
        root.ls(recursive=True)
        
        print('\nMove test2.txt to ../dir2/test3.txt:')
        path = path.move('../dir2/test3.txt')
        root.ls(recursive=True)
        
        print('\nMove test3.txt to ..:')
        path = path.move('..')
        root.ls(recursive=True)
        
        print('\n\n')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/e0a21e

Move test1.txt to test2.txt:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/e0a21e
  |
  +-dir1
    |
    +-test2.txt

Move test2.txt to ../dir2/test3.txt:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/e0a21e
  |
  +-dir1
  |
  +-dir2
    |
    +-test3.txt

Move test3.txt to ..:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/e0a21e
  |
  +-dir1
  |
  +-dir2
  |
  +-test3.txt



HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/CSrxZn/test.h5:/

Move test1.txt to test2.txt:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/CSrxZn/test.h5:/
  |
  +-dir1
    |
    +-test2.txt

Move test2.txt to ../dir2/test3.txt:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/CSrxZn/test.h5:/
  |
  +-dir1
  |
  +-dir2
    |
    +-test3.txt

Move test3.txt to ..:
+-/mntdirect/_data_id21_inhouse/wout/d

### Copy

Copying files or directories can be done with the "copy" method (alias: "cp"). The destination can be a proxy or a string. Existing destinations are replaced when "force=True".

In [10]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir1'].mkdir()['test1.txt'].mkfile(data='Hello World')
        
        print('\nCopy test1.txt to test2.txt')
        path = path.copy('test2.txt')
        root.ls(recursive=True)
        
        print('\nCopy test2.txt to ../dir2/test3.txt')
        path = path.copy('../dir2/test3.txt')
        root.ls(recursive=True)
        
        print('\nCopy test3.txt to ../dir1/test1.txt (force overwrite)')
        try:
            path = path.copy(root['dir1']['test1.txt'])
        except fs.AlreadyExists:
            path = path.copy(root['dir1']['test1.txt'],force=True)
        else:
            raise RuntimeError('Copy should not work')
        root.ls(recursive=True)
        
        print('\nMove test1.txt to test2.txt (force overwrite)')
        try:
            path = path.move('test2.txt')
        except fs.AlreadyExists:
            path = path.move('test2.txt',force=True)
        else:
            raise RuntimeError('Move should not work')
        root.ls(recursive=True)
        
        print('\n\n')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/exQwKf

Copy test1.txt to test2.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/exQwKf
  |
  +-dir1
    |
    +-test1.txt
    |
    +-test2.txt

Copy test2.txt to ../dir2/test3.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/exQwKf
  |
  +-dir1
  | |
  | +-test1.txt
  | |
  | +-test2.txt
  |
  +-dir2
    |
    +-test3.txt

Copy test3.txt to ../dir1/test1.txt (force overwrite)
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/exQwKf
  |
  +-dir1
  | |
  | +-test1.txt
  | |
  | +-test2.txt
  |
  +-dir2
    |
    +-test3.txt

Move test1.txt to test2.txt (force overwrite)
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/exQwKf
  |
  +-dir1
  | |
  | +-test2.txt
  |
  +-dir2
    |
    +-test3.txt



HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/019Yy4/test.h5:/

Copy test1.txt to test2.txt
+-/mntdirect/_da

### Links

Hard and soft links are supported. The difference is that when deleting the link destination, the hard link still exists (file is not actually deleted) while the soft link is broken.

In [11]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir1'].mkdir()['file.txt'].mkfile(data='Hello World')
        soft = root['soft.txt'].link(path)
        hard = root['hard.txt'].link(path,soft=False)
        soft2 = root['soft2.txt'].link('hard.txt')
        assert(soft.read()=='Hello World')
        assert(soft2.read()=='Hello World')
        assert(hard.read()=='Hello World')
        root.ls(recursive=True,stats=hdf5)
        
        print('\nRemove file.txt')
        soft.linkdest().remove(recursive=True)
        assert(soft.lexists)
        assert(not soft.exists)
        assert(soft2.read()=='Hello World')
        assert(hard.read()=='Hello World')
        root.ls(recursive=True,stats=hdf5)
        
        print('\n\n')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/W5Eain
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/W5Eain
  |
  +-dir1
  | |
  | +-file.txt
  |
  +-hard.txt
  |
  +-soft.txt ---> dir1/file.txt
  |
  +-soft2.txt ---> hard.txt

Remove file.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/W5Eain
  |
  +-dir1
  |
  +-hard.txt
  |
  +-soft.txt -/-> dir1/file.txt
  |
  +-soft2.txt ---> hard.txt



HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/5cadnr/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/5cadnr/test.h5:/
  |
  +-dir1
  | |
  | +-file.txt = Hello World
  |
  +-hard.txt = Hello World
  |
  +-soft.txt ---> dir1/file.txt = Hello World
  |
  +-soft2.txt ---> hard.txt = Hello World

Remove file.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/5cadnr/test.h5:/
  |
  +-dir1
  |
  +-hard.txt = Hello World
  |
  +-soft.txt -/-> dir1/file.tx

### Cross-device links

HDF5 also supports cross-device links (called external links):

In [12]:
with temppath(hdf5=True) as root1:
    root2 = h5fs.Path(root1.device.parent['external.h5']).mkdir()
    path = root1['dir1'].mkdir()['a'].mkfile(data='Hello World')
    
    lnk = root2['alnk'].link(path)
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    path = lnk.linkdest().rename('b')
    
    print('\n\nLink destination renamed:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    lnk = root2['blnk'].link(path)
    root2['alnk'].remove()
    
    print('\n\nLink to renamed destination:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    lnk.linkdest().copy(root2['b'])
    
    print('\n\nCopy destination to local tree:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    lnk.linkdest().move(root2['c'])
    
    print('\n\nMove destination to local tree:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    root2['b'].move(root1['dir1'])
    
    print('\n\nMove from local tree to destination:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    root2['blnk'].remove(recursive=True)
    
    print('\n\nRemove link recursively:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/test.h5:/
  |
  +-dir1
    |
    +-a = Hello World

+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/external.h5:/
  |
  +-alnk ---> /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/test.h5:/dir1/a = Hello World


Link destination renamed:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/test.h5:/
  |
  +-dir1
    |
    +-b = Hello World

+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/external.h5:/
  |
  +-alnk -/-> /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/test.h5:/dir1/a


Link to renamed destination:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/RNvTQv/test.h5:/
  |
  +-dir1
    |
    +-b = Hello World

+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/R

## Devices

If a proxy is derived from another proxy, they share the same device. Therefore if the device is already opened by one proxy, it will be open for the other proxy. Re-opening is allowed but does not actually do anything (no reference counting either).

In [13]:
with temppath(hdf5=True) as root1:
    # Shared device
    deviceid = id(root1.device)
    path = root1['dir1']['dir2'].mkdir()
    assert(deviceid==id(path.device))
    txtfile = path['file.txt'].open(data='Hello World')
    assert(deviceid==id(path.device))
    
    # Shared device is opened once
    with root1['dir1'].open() as grp1:
        assert(root1['dir1']['dir2'].device.isopen())
        with root1['dir1']['dir2'].open() as grp2:
            assert(grp1['dir2']==grp2)
    
    # Same device but not shared
    root2 = h5fs.Path(root1.location,mode='r')
    assert(deviceid!=id(root2.device))

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/nOgk8Z/test.h5:/


## Access modes

Files can be opened in whatever mode the underlying file system supports (read, read/write, append, truncate,...). The default mode is read/write and create when not existing (local file system: mode='a+', HDF5 file system: mode='a'). This default can be changed when opening the file. A proxy which has a device associated with it (like HDF5 proxies) will take the device's access mode by default.

In [40]:
with temppath(hdf5=True) as root:
    print('Default proxy mode: {}'.format(root.openparams['mode']))
    rootreadonly = h5fs.Path(root.location,mode='r')
    print('Read-only proxy mode: {}'.format(rootreadonly.openparams['mode']))

    # Write a file
    txtfile = root['file.txt'].mkfile(data='123',mode='x')
    txtfile = root['file.txt'].mkfile(data='abcd',mode='w')
    assert(txtfile.read()=='abcd')

    # Try to write a file in the wrong mode (default proxy):
    for mode in ['r','x']:
        try:
            txtfile = root['file.txt'].mkfile(data='123',mode=mode)
        except (IOError,fs.AlreadyExists):
            pass
        else:
            raise RuntimeError('Should throw an error')    
            
    # Try to write a file in the wrong mode (read-only proxy):
    for mode in ['r','x','w','a']:
        try:
            txtfile = rootreadonly['file.txt'].mkfile(data='123',mode=mode)
        except (IOError,fs.AlreadyExists):
            pass
        else:
            raise RuntimeError('Should throw an error')
            
    assert(txtfile.read()=='abcd')

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/b4DTGW/test.h5:/
Default proxy mode: a
Read-only proxy mode: r


## Nexus

Nexus proxies inherit from HDF5 proxies with additional functionality to enforce Nexus standards.

In [49]:
with temppath(nexus=True) as root:
    root.nxentry(name='test0001')
    entry = root.new_nxentry()
    subentry = entry.nxsubentry('subentry')
    instrument = entry.nxinstrument()
    det1 = entry.nxdetector('xia')
    det2 = entry.nxdetector('pco')
    measurement = entry.measurement()
    nxmonochromator = entry.nxmonochromator()
    positioners = entry.positioners()
    application = entry.application('xrf')
    root.ls(recursive=True,stats=True)

Nexus root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/NJiWFl/test.nx:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/NJiWFl/test.nx:/
  | @HDF5_Version = 1.10.2
  | @creator = spectrocrunch
  | @file_name = /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/NJiWFl/test.nx
  | @NX_class = NXroot
  | @file_time = 2018-12-05T12:44:10.017429
  | @h5py_version = 2.8.0
  | @file_update_time = 2018-12-05T12:44:11.091453
  |
  +-test0001
  | | @NX_class = NXentry
  | |
  | +-end_time = 2018-12-05T12:44:10.081044
  | |
  | +-start_time = 2018-12-05T12:44:10.077062
  |
  +-test0002
    | @NX_class = NXentry
    |
    +-end_time = 2018-12-05T12:44:11.091453
    |
    +-instrument
    | | @NX_class = NXinstrument
    | |
    | +-monochromator
    | |   @NX_class = NXmonochromator
    | |
    | +-pco
    | |   @NX_class = NXdetector
    | |
    | +-positioners
    | |   @NX_class = NXcollection
    | |
    | +-xia
    |     @NX_class = NXdet

Apart from the creation of Nexus groups, Nexus fields and attributes are currently neither enforced, restricted or dealth with automatically. For example:

In [45]:
with temppath(nexus=True) as root:
    entry = root.new_nxentry()
    instrument = entry.nxinstrument()
    
    name = instrument['name']
    if not name.exists:
        name = name.mkfile(data='ID21 Scanning X-ray Microscope')
        name.update_stats(short_name='ID21')
    
    root.ls(stats=True,recursive=True)

Nexus root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/PQpmYZ/test.nx:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/PQpmYZ/test.nx:/
  | @HDF5_Version = 1.10.2
  | @creator = spectrocrunch
  | @file_name = /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/PQpmYZ/test.nx
  | @NX_class = NXroot
  | @file_time = 2018-12-05T12:22:41.059103
  | @h5py_version = 2.8.0
  | @file_update_time = 2018-12-05T12:22:41.241059
  |
  +-entry0001
    | @NX_class = NXentry
    |
    +-end_time = 2018-12-05T12:22:41.241059
    |
    +-instrument
    | | @NX_class = NXinstrument
    | |
    | +-name = ID21 Scanning X-ray Microscope
    |     @short_name = ID21
    |
    +-start_time = 2018-12-05T12:22:41.116874


Default plotting:

In [50]:
with temppath(nexus=True) as root:
    entry = root.nxentry(name='entry.1')
    entry = root.new_nxentry()
    
    # Add axes
    positioners = entry.positioners()
    shape = (2,20,30)
    positioners.add_axis('y',range(shape[1]),units='um',title='vertical')
    positioners.add_axis('x',range(shape[2]),units='um',title='horizontal')
    energy = 'energy',range(5,5+shape[0]),{'units':'keV','title':'energy'}
    
    # Add plotable groups (NXdata)
    for i in [1,2]:
        nxdata = entry.nxdata('plot'+str(i))
        # Add signals
        for signal in ['A','B','C']:
            signal = nxdata.add_signal(name=signal,dtype=float,shape=shape)
        # Add axes (by name or by value)
        nxdata.set_axes(energy,'y','x')
    
    assert('energy' in positioners)
    
    # Select default plot
    plotselect = entry['plotselect'].link(entry['plot1'])
    plotselect.mark_default()
    assert(root.default==plotselect)
    assert(plotselect.signal.name=='C')
    plotselect.default_signal('B')
    assert(plotselect.signal.name=='B')
    
    root.ls(recursive=True,stats=True)

Nexus root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/rTJpkR/test.nx:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/rTJpkR/test.nx:/
  | @HDF5_Version = 1.10.2
  | @creator = spectrocrunch
  | @default = entry.2
  | @file_name = /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/rTJpkR/test.nx
  | @NX_class = NXroot
  | @file_time = 2018-12-05T12:44:25.852062
  | @h5py_version = 2.8.0
  | @file_update_time = 2018-12-05T12:44:27.371590
  |
  +-entry.1
  | | @NX_class = NXentry
  | |
  | +-end_time = 2018-12-05T12:44:25.908688
  | |
  | +-start_time = 2018-12-05T12:44:25.904699
  |
  +-entry.2
    | @default = plotselect
    | @NX_class = NXentry
    |
    +-end_time = 2018-12-05T12:44:27.371590
    |
    +-instrument
    | | @NX_class = NXinstrument
    | |
    | +-positioners
    |   | @NX_class = NXcollection
    |   |
    |   +-energy = int64 (2)
    |   |   @units = keV
    |   |   @long_name = energy
    |   |
    |   +-x =

Results of data processing:

In [57]:
with temppath(nexus=True) as root:
    entry = root.new_nxentry()
    
    # Results of a process with certian parameters
    parameters = {'a':1,'b':2}
    process,existed = entry.nxprocess(name='proc1',parameters=parameters)
    assert(existed==False)
    assert(process.config.read()==parameters)
    process.results['result1'].write(data=range(10))
    process.results['result2'].write(data=range(20))
    
    # Retrieve the first process again
    process,existed = entry.nxprocess(name='proc1',parameters=parameters)
    assert(existed)
    assert(process.config.read()==parameters)

    # Results of a second process based on the first
    process2,existed = entry.nxprocess(name='proc2',dependencies=[process],parameters=parameters)
    assert(existed==False)
    assert(next(iter(process2.dependencies)).linkdest()==process)
    
    root.ls(recursive=True,stats=True)

Nexus root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/zULi0a/test.nx:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/zULi0a/test.nx:/
  | @HDF5_Version = 1.10.2
  | @creator = spectrocrunch
  | @file_name = /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/zULi0a/test.nx
  | @NX_class = NXroot
  | @file_time = 2018-12-05T12:53:46.243290
  | @h5py_version = 2.8.0
  | @file_update_time = 2018-12-05T12:53:47.309329
  |
  +-entry0001
    | @NX_class = NXentry
    |
    +-end_time = 2018-12-05T12:53:47.309329
    |
    +-proc1
    | | @NX_class = NXprocess
    | |
    | +-configuration
    | | | @NX_class = NXnote
    | | |
    | | +-data = {"a": 1, "b": 2}
    | | |   @type = application/json
    | | |
    | | +-date = 2018-12-05T12:53:46.415446
    | |
    | +-date = 2018-12-05T12:53:46.666263
    | |
    | +-program = spectrocrunch
    | |
    | +-results
    | | | @NX_class = NXcollection
    | | |
    | | +-checksum = 5c4be2c2c